首先要弄清楚的是Andrew讲的RNN/LSTM模型和Keras提供的模型是不一样的，主要区别是在outlayer的处理上，Andrew/李宏毅将的RNN/LSTM cell中都包含outputlayer,但是Keras值提供hidden layer.  

# RNN/LSTM in Keras

Keras中的RNN模型
keras.layers.RNN(cell, return_sequences=False, return_state=False, go_backwards=False, stateful=False, unroll=False)

Keras的LSTM模型
keras.layers.LSTM(units, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0, implementation=1, return_sequences=False, return_state=False, go_backwards=False, stateful=False, unroll=False)

其中的cell/units代表隐藏层的neuros/nodes的个数  
首先理解的是RNN的输入，是个 3D的，格式是(batch_size, timesteps, features).  
输出受参数的影响.  
[Ref](https://machinelearningmastery.com/return-sequences-and-return-states-for-lstms-in-keras/)

The output of an LSTM cell or layer of cells is called the hidden state.  
If _return_sequences_ is **True**，返回的是也是一个序列，每个序列的值为对应cell的hidden state  
If _return_sequences_ is **False**, 返回的是一个值，即最后一个timestep对应的cell的hidden state,即上面序列的最后一个值  
* That return sequences return the hidden state output for each input time step.
* That return state returns the hidden state output and cell state for the last input time step.
* That return sequences and return state can be used at the same time.

In [1]:
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from numpy import array
# define model
inputs1 = Input(shape=(3, 1))
lstm1, state_h, state_c = LSTM(1, return_sequences=True, return_state=True)(inputs1)
model = Model(inputs=inputs1, outputs=[lstm1, state_h, state_c])
# define input data
data = array([0.1, 0.2, 0.3]).reshape((1,3,1))
# make and show prediction
print(model.predict(data))

Using TensorFlow backend.


[array([[[-0.01934982],
        [-0.0542377 ],
        [-0.099271  ]]], dtype=float32), array([[-0.099271]], dtype=float32), array([[-0.2054972]], dtype=float32)]


The layer returns the hidden state for each input time step, then separately, the hidden state output for the last time step and the cell state for the last input time step.  
lstm1就是返回的sequence, state_h是最后一步的hidden state, state_c是最后一步的cell state.






# 根据返回的是序列还是非序列，对构建不同的RNN model影响非常大

参考 连接 https://medium.com/@daniel820710/%E5%88%A9%E7%94%A8keras%E5%BB%BA%E6%A7%8Blstm%E6%A8%A1%E5%9E%8B-%E4%BB%A5stock-prediction-%E7%82%BA%E4%BE%8B-1-67456e0a0b

## 1. One vs One model

In [2]:
def buildOneToOneModel(shape):
  model = Sequential()
  model.add(LSTM(10, input_length=shape[1], input_dim=shape[2],return_sequences=True))
  # output shape: (1, 1)
  model.add(TimeDistributed(Dense(1)))    # or use model.add(Dense(1))
  model.compile(loss="mse", optimizer="adam")
  model.summary()
  return model

# 2. Many to One model

In [3]:
def buildManyToOneModel(shape):
  model = Sequential()
  model.add(LSTM(10, input_length=shape[1], input_dim=shape[2]))
  # output shape: (1, 1)
  model.add(Dense(1))
  model.compile(loss="mse", optimizer="adam")
  model.summary()
  return model

上面没有传return_sequences,默认是False,所以输出是最后一个cell的hidden state，(1, 10)，再加上一个Dense layer(FC),转成（1,1）

# 3. One to Many model

In [5]:
def buildOneToManyModel(shape):
  model = Sequential()
  model.add(LSTM(10, input_length=shape[1], input_dim=shape[2]))
  # output shape: (5, 1)
  model.add(Dense(1))
  model.add(RepeatVector(5))
  model.compile(loss="mse", optimizer="adam")
  model.summary()
  return model

这里LSTM的输出是(1,10), 通过Dense后变为(1,1)，然后用了个 RepeatVector,扩展成(5,1),但是个人感觉只是复制了5份，维度匹配了，但是每个值是一样的。  
这里是关于这个的一个[讨论](https://stackoverflow.com/questions/51749404/how-to-connect-lstm-layers-in-keras-repeatvector-or-return-sequence-true)  
![discussion](repeat_vector.jpg)

# 4. Many to Many model(with the same length)

In [7]:
def buildManyToManyModel(shape):
  model = Sequential()
  model.add(LSTM(10, input_length=shape[1], input_dim=shape[2], return_sequences=True))
  # output shape: (5, 1)
  model.add(TimeDistributed(Dense(1)))
  model.compile(loss="mse", optimizer="adam")
  model.summary()
  return model

首先是return_sequences=True，返回一个序列，得到(timesteps, 10)，然后通过TimeDistributed(Dense(1))，转化成(timesteps, 1)，注意这里的每个Dense Layer的参数都是一样的。关于TimeDistributed(Dense(1))， 详细请参考[这里](https://machinelearningmastery.com/timedistributed-layer-for-long-short-term-memory-networks-in-python/), 一个关键就是每个的Dense Layer的参数是一样的。

# 5. Many to Many model(with different lengths)
文中让参考另一篇文章。  
我在网上找到另一篇讨论，[Here](https://github.com/keras-team/keras/issues/5203)  
其中给出的结论是不用Keras，而是用的另外一个sequence2sequence,https://github.com/farizrahman4u/seq2seq 

# Conclusion.   
个人感觉Keras灵活度有限，有些RNN模型很难用Keras去实现，而Andrew讨论的模型中每个cell通常含有输出层，即Wya,自己实现才能达到想要的版本.
观察Andrew的课后作业，可以看出其自己实现的RNN cell中输出层。